## Problem 1 – Exact Currency Invoice (no float leaks!)

You are given a list of items:

```
items = [
    {"name": "Book",   "unit_price": "12.99", "quantity": 2},
    {"name": "Pen",    "unit_price": "1.20",  "quantity": 10},
    {"name": "Coffee", "unit_price": "2.35",  "quantity": 1},
]
tax_rate = "0.07"  # 7%
```

Tasks:
1. Compute line net, tax, and gross amounts using `Decimal` only.
2. Sum totals.
3. Round all monetary values to 2 decimals using bankers rounding.
4. Compare with a float-based calculation.

**Best practice:** Never construct `Decimal` from a float!

In [1]:
from decimal import Decimal, ROUND_HALF_EVEN, getcontext

items = [
    {"name": "Book",   "unit_price": "12.99", "quantity": 2},
    {"name": "Pen",    "unit_price": "1.20",  "quantity": 10},
    {"name": "Coffee", "unit_price": "2.35",  "quantity": 1},
]
tax_rate = Decimal("0.07")

CENT = Decimal("0.01")

def to_money(d: Decimal) -> Decimal:
    return d.quantize(CENT, rounding=ROUND_HALF_EVEN)

invoice_lines = []
total_net = Decimal("0")
total_tax = Decimal("0")
total_gross = Decimal("0")

for item in items:
    unit_price = Decimal(item["unit_price"])
    qty = item["quantity"]

    net = unit_price * qty
    tax = net * tax_rate
    gross = net + tax

    net = to_money(net)
    tax = to_money(tax)
    gross = to_money(gross)

    total_net += net
    total_tax += tax
    total_gross += gross

    invoice_lines.append({"name": item["name"], "net": net, "tax": tax, "gross": gross})

print("Lines:")
for line in invoice_lines:
    print(line)

print("Totals:")
print("Net:  ", total_net)
print("Tax:  ", total_tax)
print("Gross:", total_gross)


Lines:
{'name': 'Book', 'net': Decimal('25.98'), 'tax': Decimal('1.82'), 'gross': Decimal('27.80')}
{'name': 'Pen', 'net': Decimal('12.00'), 'tax': Decimal('0.84'), 'gross': Decimal('12.84')}
{'name': 'Coffee', 'net': Decimal('2.35'), 'tax': Decimal('0.16'), 'gross': Decimal('2.51')}
Totals:
Net:   40.33
Tax:   2.82
Gross: 43.15


### Float comparison (expected to differ by small cents)

In [2]:
items_float = [
    {"name": x["name"], "unit_price": float(x["unit_price"]), "quantity": x["quantity"]}
    for x in items
]
tax_rate_f = float("0.07")

total_gross_f = 0.0
for item in items_float:
    net = item["unit_price"] * item["quantity"]
    tax = net * tax_rate_f
    gross = net + tax
    total_gross_f += round(gross, 2)

print("Float total gross:", total_gross_f)


Float total gross: 43.15


## Problem 2 – Compare HALF_EVEN vs HALF_UP Rounding

Given values:
```
values = ["1.245", "1.255", "2.335", "2.345"]
```

Tasks:
1. Implement HALF_EVEN and HALF_UP rounding helpers.
2. Round each number to 2 decimal places in both modes.
3. Display the results in a table.

In [3]:
from decimal import Decimal, ROUND_HALF_EVEN, ROUND_HALF_UP

values = ["1.245", "1.255", "2.335", "2.345"]

def round_half_even(d: Decimal, places: int = 2) -> Decimal:
    exponent = Decimal("1").scaleb(-places)
    return d.quantize(exponent, rounding=ROUND_HALF_EVEN)

def round_half_up(d: Decimal, places: int = 2) -> Decimal:
    exponent = Decimal("1").scaleb(-places)
    return d.quantize(exponent, rounding=ROUND_HALF_UP)

print(f"{'original':>10}  {'half_even':>10}  {'half_up':>10}")
for s in values:
    d = Decimal(s)
    he = round_half_even(d, 2)
    hu = round_half_up(d, 2)
    print(f"{s:>10}  {he:>10}  {hu:>10}")


  original   half_even     half_up
     1.245        1.24        1.25
     1.255        1.26        1.26
     2.335        2.34        2.34
     2.345        2.34        2.35


## Problem 3 – Local Context Precision: 1/7 and √2

Tasks:
1. Write `approx_fraction(n, d, prec)` that sets precision locally.
2. Compute 1/7 at precisions 5, 10, 20.
3. Compute sqrt(2) at precisions 5, 10, 20 using `Decimal.sqrt()`.

Use `decimal.localcontext()` so global settings are unchanged.

In [4]:
import decimal
from decimal import Decimal, localcontext

def approx_fraction(numerator: int, denominator: int, prec: int) -> Decimal:
    with localcontext() as ctx:
        ctx.prec = prec
        num = Decimal(str(numerator))
        den = Decimal(str(denominator))
        return num / den

def approx_sqrt2(prec: int) -> Decimal:
    with localcontext() as ctx:
        ctx.prec = prec
        return Decimal("2").sqrt()

print("1/7 approximations:")
for p in (5, 10, 20):
    print(f"prec={p}: {approx_fraction(1, 7, p)}")

print("\n√2 approximations:")
for p in (5, 10, 20):
    print(f"prec={p}: {approx_sqrt2(p)}")


1/7 approximations:
prec=5: 0.14286
prec=10: 0.1428571429
prec=20: 0.14285714285714285714

√2 approximations:
prec=5: 1.4142
prec=10: 1.414213562
prec=20: 1.4142135623730950488


## Problem 4 – Percentage Changes with Fixed Precision

Given daily closing prices:
```
["1.0990", "1.1008", "1.0850", "1.0818"]
```

Tasks:
1. Convert to Decimals.
2. Compute percentage change from one day to the next:
   
   (new - old) / old × 100

3. Round to 4 decimal places using HALF_EVEN.
4. Print the results.

In [5]:
from decimal import Decimal, ROUND_HALF_EVEN

price_strings = ["1.0990", "1.1008", "1.0850", "1.0818"]
prices = [Decimal(s) for s in price_strings]

def pct_changes(prices, places: int = 4):
    if len(prices) < 2:
        return []

    exponent = Decimal("1").scaleb(-places)
    changes = []

    for prev, curr in zip(prices, prices[1:]):
        pct = (curr - prev) / prev * Decimal("100")
        pct = pct.quantize(exponent, rounding=ROUND_HALF_EVEN)
        changes.append(pct)

    return changes

changes = pct_changes(prices, places=4)

print("Prices:")
for p in prices:
    print(p)

print("\nDaily % changes (rounded to 4 dp):")
for c in changes:
    print(f"{c} %")


Prices:
1.0990
1.1008
1.0850
1.0818

Daily % changes (rounded to 4 dp):
0.1638 %
-1.4353 %
-0.2949 %


## Problem 5 – Safe Decimal Sum from Mixed Data Types

Given:
```
raw_values = [0.1, "0.2", Decimal("0.3"), 1, "2.50", 3.75]
```

Tasks:
1. Write `to_decimal_safe(x)`:
   - Floats → convert via `str()` and warn.
   - Strings → Decimal
   - Ints → Decimal
   - Decimals → untouched
2. Write `sum_decimals(values)` that uses it.
3. Compute the total.

In [6]:
from decimal import Decimal

raw_values = [0.1, "0.2", Decimal("0.3"), 1, "2.50", 3.75]

def to_decimal_safe(x) -> Decimal:
    if isinstance(x, Decimal):
        return x
    elif isinstance(x, int):
        return Decimal(x)
    elif isinstance(x, str):
        return Decimal(x)
    elif isinstance(x, float):
        print(f"Warning: converting float {x!r} to Decimal via str")
        return Decimal(str(x))
    else:
        raise TypeError(f"Unsupported type for Decimal conversion: {type(x)!r}")

def sum_decimals(values) -> Decimal:
    total = Decimal("0")
    for v in values:
        total += to_decimal_safe(v)
    return total

total = sum_decimals(raw_values)
print("Total as Decimal:", total)


Total as Decimal: 7.85
